# Visualizing Metop and MSG data on the same grid
## Summary
In this tutorial, we well look at how we can put both Metop and MSG data on the same grid

## First step, loading the data
Loading the data is done in the same way as presented in earlier tutorial

## Exercise 6.1: Getting the data on the same grid
Create a `Scene` object for the Metop data and another one for the MSG data. Load the `day_microphysics` RGB composite from MSG and the 10.8µm channel from Metop. Resample both to the EastEurope area.

In [ ]:
from satpy import Scene, find_files_and_readers
from datetime import datetime

metop_files = find_files_and_readers(base_dir='./sample_data',
                                     reader='avhrr_eps_l1b',
                                     start_time=datetime(2018, 10, 3 ,10, 0),
                                     end_time=datetime(2018, 10, 3 ,10, 20))
scn_metop = Scene(filenames=metop_files)
scn_metop.load([10.8])

In [ ]:
msg_files = find_files_and_readers(base_dir='./sample_data',
                                   reader='native_msg')
scn = Scene(filenames=msg_files)
scn_msg = Scene(filenames=msg_files)
scn_msg.load(['day_microphysics'])

In [ ]:
newscn_metop = scn_metop.resample('EastEurope')
newscn_metop.show(10.8)

In [ ]:
newscn_msg = scn_msg.resample('EastEurope')
newscn_msg.show('day_microphysics')

## Exercise 6.2:  Manipulating images
Now we want to put both datasets onto one image.

In [ ]:
from satpy.writers import get_enhanced_image
import xarray as xr
from trollimage.xrimage import XRImage

msg_image = get_enhanced_image(newscn_msg['day_microphysics'])

metop_image = get_enhanced_image(newscn_metop[10.8])

color_array = xr.concat((metop_image.data, metop_image.data, metop_image.data), 'bands')
color_array['bands'] = ['R', 'G', 'B']
final_array = msg_image.data.where(color_array.isnull(), color_array.data)


XRImage(final_array)


## The `Multiscene` class
In order to manipulate multiple scenes at once, in some circumstances, it maybe advantageous to make use of satpy's `Multiscene` class. Let's create a multiscene instance, load some data and resample the data to the `EastEurope` area

In [ ]:
from satpy import MultiScene
mscn = MultiScene([scn_msg, scn_metop])
mscn.load(['overview_sun'])
new_mscn = mscn.resample('EastEurope')
blended_scene = new_mscn.blend()
blended_scene.show('overview_sun', overlay={'coast_dir': '/home/a001673/data/shapes/', 'color': (255, 0, 0), 'resolution': 'i'})